# 📊 QuantBT 기본 전략 튜토리얼

이 튜토리얼에서는 QuantBT에서 제공하는 기본 전략들을 단계별로 알아보겠습니다.

## 📋 목차

1. [전략 기본 개념](#전략-기본-개념)
2. [바이 앤 홀드 전략](#바이-앤-홀드-전략)
3. [이동평균 교차 전략](#이동평균-교차-전략)
4. [RSI 전략](#rsi-전략)
5. [랜덤 전략](#랜덤-전략)
6. [전략 비교 및 실행](#전략-비교-및-실행)

---


## 🎯 전략 기본 개념

QuantBT의 모든 전략은 `TradingStrategy` 클래스를 상속받아 구현됩니다.

### 핵심 특징
- **지표 사전 계산**: 백테스팅 시작 전 모든 지표를 미리 계산
- **단순 신호 생성**: 백테스팅 중에는 계산된 지표값과 단순 비교로 신호 생성
- **룩어헤드 바이어스 방지**: 각 시점에서 과거 데이터만 접근 가능

### 필수 구현 메서드
- `_compute_indicators_for_symbol()`: 심볼별 지표 사전 계산
- `generate_signals()`: 신호 생성 로직


In [1]:
# 프로젝트 루트를 Python 경로에 추가
import sys
import os
from pathlib import Path

# 현재 노트북의 위치에서 프로젝트 루트 찾기
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # examples 폴더에서 실행하는 경우
    project_root = current_dir.parent.parent
else:
    # 프로젝트 루트에서 실행하는 경우
    project_root = current_dir

# 프로젝트 루트를 Python 경로에 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 필요한 모듈 가져오기
from typing import List, Dict, Any, Optional
import polars as pl

from quantbt.core.interfaces.strategy import TradingStrategy, BacktestContext
from quantbt.core.entities.market_data import MarketDataBatch
from quantbt.core.entities.order import Order, OrderType, OrderSide
from quantbt.core.entities.trade import Trade
print("✅ 모든 QuantBT 모듈이 성공적으로 가져와졌습니다!")

✅ 모든 QuantBT 모듈이 성공적으로 가져와졌습니다!


---

## 1️⃣ 바이 앤 홀드 전략

가장 단순한 전략으로, **한 번 매수 후 계속 보유**하는 전략입니다.

### 🎯 전략 특징
- 초기에 한 번만 매수
- 이후 계속 보유 (매도 없음)
- 지표 계산 불필요
- 시장 전체 성장에 베팅


In [2]:
class BuyAndHoldStrategy(TradingStrategy):
    """바이 앤 홀드 전략 - 한 번 매수 후 보유"""
    
    def __init__(self):
        super().__init__(
            name="BuyAndHoldStrategy",
            config={},
            position_size_pct=1.0,  # 전체 자본으로 매수
            max_positions=10
        )
        self.bought_symbols: set[str] = set()  # 이미 매수한 심볼들 추적
        print("🏠 바이 앤 홀드 전략이 초기화되었습니다.")
        
    def _compute_indicators_for_symbol(self, symbol_data: pl.DataFrame) -> pl.DataFrame:
        """바이 앤 홀드는 지표 불필요 - 원본 데이터 그대로 반환"""
        print(f"   📊 {symbol_data['symbol'][0]} 심볼 데이터 준비 완료 (지표 계산 없음)")
        return symbol_data
    
    def generate_signals(self, data: MarketDataBatch) -> List[Order]:
        """신호 생성 - 한 번만 매수"""
        orders = []
        
        if not self.context:
            return orders
        
        # 아직 매수하지 않은 심볼 중 하나만 선택하여 매수
        for symbol in data.symbols:
            if symbol not in self.bought_symbols:
                current_price = self.get_current_price(symbol, data)
                if current_price and current_price > 0:
                    # 현재 포트폴리오 가치의 일정 비율로 매수
                    current_portfolio_value = self.get_portfolio_value()
                    position_value = current_portfolio_value / len(self.context.symbols) * 0.8  # 80%만 사용
                    quantity = position_value / current_price
                    
                    # 최소 수량 확인
                    if quantity > 0.01:
                        order = Order(
                            symbol=symbol,
                            side=OrderSide.BUY,
                            quantity=quantity,
                            order_type=OrderType.MARKET
                        )
                        orders.append(order)
                        self.bought_symbols.add(symbol)
                        print(f"🛒 {symbol} 매수 주문: {quantity:.4f}주 @ ${current_price:.2f}")
                        break  # 한 번에 하나씩만 매수
        
        return orders

# 전략 인스턴스 생성
buy_hold = BuyAndHoldStrategy()
print(f"📋 전략명: {buy_hold.name}")
print(f"💰 포지션 크기: {buy_hold.position_size_pct * 100}%")
print(f"📈 최대 포지션 수: {buy_hold.max_positions}")


🏠 바이 앤 홀드 전략이 초기화되었습니다.
📋 전략명: BuyAndHoldStrategy
💰 포지션 크기: 100.0%
📈 최대 포지션 수: 10


---

## 2️⃣ 이동평균 교차 전략

**골든 크로스**와 **데드 크로스**를 이용한 클래식한 기술적 분석 전략입니다.

### 🎯 전략 로직
- **골든 크로스**: 단기 MA > 장기 MA 될 때 **매수**
- **데드 크로스**: 단기 MA < 장기 MA 될 때 **매도**
- 트렌드 추종 전략의 대표격

### 📊 기본 설정
- 단기 이동평균: 10일
- 장기 이동평균: 30일


In [3]:
class SimpleMovingAverageCrossStrategy(TradingStrategy):
    """단순 이동평균 교차 전략 - 지표 사전 계산 버전"""
    
    def __init__(self, short_window: int = 10, long_window: int = 30):
        super().__init__(
            name="SimpleMovingAverageCrossStrategy",
            config={
                "short_window": short_window,
                "long_window": long_window
            },
            position_size_pct=0.2,  # 20%씩 포지션
            max_positions=5
        )
        self.short_window = short_window
        self.long_window = long_window
        self.indicator_columns = [f"sma_{short_window}", f"sma_{long_window}"]
        print(f"📈 이동평균 교차 전략 초기화 (단기: {short_window}일, 장기: {long_window}일)")
        
    def _compute_indicators_for_symbol(self, symbol_data: pl.DataFrame) -> pl.DataFrame:
        """심볼별 이동평균 지표 계산"""
        # 시간순 정렬 확인
        data = symbol_data.sort("timestamp")
        
        # 단순 이동평균 계산
        short_sma = self.calculate_sma(data["close"], self.short_window)
        long_sma = self.calculate_sma(data["close"], self.long_window)
        
        print(f"   📊 {data['symbol'][0]} - SMA({self.short_window}), SMA({self.long_window}) 계산 완료")
        
        # 지표 컬럼 추가
        return data.with_columns([
            short_sma.alias(f"sma_{self.short_window}"),
            long_sma.alias(f"sma_{self.long_window}")
        ])
    
    def generate_signals(self, data: MarketDataBatch) -> List[Order]:
        """신호 생성 - 단순 이동평균 교차 확인"""
        orders = []
        
        if not self.context:
            return orders
        
        for symbol in data.symbols:
            current_price = self.get_current_price(symbol, data)
            if not current_price:
                continue
            
            # 현재 지표 값 조회
            short_ma = self.get_indicator_value(symbol, f"sma_{self.short_window}", data)
            long_ma = self.get_indicator_value(symbol, f"sma_{self.long_window}", data)
            
            if short_ma is None or long_ma is None:
                continue
            
            # 이전 지표 값 조회 (골든/데드 크로스 확인을 위해)
            symbol_data = data.get_symbol_data(symbol)
            if symbol_data.height < 2:
                continue
            
            prev_row = symbol_data.row(-2, named=True)
            prev_short_ma = prev_row.get(f"sma_{self.short_window}")
            prev_long_ma = prev_row.get(f"sma_{self.long_window}")
            
            if prev_short_ma is None or prev_long_ma is None:
                continue
            
            # 골든 크로스 (매수 신호)
            if prev_short_ma <= prev_long_ma and short_ma > long_ma:
                portfolio_value = self.get_portfolio_value()
                quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
                
                if quantity > 0:
                    order = Order(
                        symbol=symbol,
                        side=OrderSide.BUY,
                        quantity=quantity,
                        order_type=OrderType.MARKET
                    )
                    orders.append(order)
                    print(f"🌟 골든 크로스 - {symbol} 매수 신호! SMA({self.short_window}): {short_ma:.2f} > SMA({self.long_window}): {long_ma:.2f}")
            
            # 데드 크로스 (매도 신호)
            elif prev_short_ma >= prev_long_ma and short_ma < long_ma:
                # 현재 포지션이 있다면 매도
                current_positions = self.get_current_positions()
                if symbol in current_positions and current_positions[symbol] > 0:
                    order = Order(
                        symbol=symbol,
                        side=OrderSide.SELL,
                        quantity=current_positions[symbol],
                        order_type=OrderType.MARKET
                    )
                    orders.append(order)
                    print(f"💀 데드 크로스 - {symbol} 매도 신호! SMA({self.short_window}): {short_ma:.2f} < SMA({self.long_window}): {long_ma:.2f}")
        
        return orders

# 전략 인스턴스 생성 및 테스트
sma_strategy = SimpleMovingAverageCrossStrategy(short_window=5, long_window=20)
print(f"📋 전략명: {sma_strategy.name}")
print(f"⚙️ 설정: {sma_strategy.config}")
print(f"💰 포지션 크기: {sma_strategy.position_size_pct * 100}%")
print(f"📊 필요 지표: {sma_strategy.indicator_columns}")


📈 이동평균 교차 전략 초기화 (단기: 5일, 장기: 20일)
📋 전략명: SimpleMovingAverageCrossStrategy
⚙️ 설정: {'short_window': 5, 'long_window': 20}
💰 포지션 크기: 20.0%
📊 필요 지표: ['sma_5', 'sma_20']


---

## 3️⃣ RSI 전략

**상대강도지수(RSI)**를 이용한 평균회귀 전략입니다.

### 🎯 전략 로직
- **과매도 구간(RSI < 30)**: 매수 신호
- **과매수 구간(RSI > 70)**: 매도 신호
- 가격이 극단적으로 움직일 때 반대 방향으로 베팅

### 📊 기본 설정
- RSI 기간: 14일
- 과매도 기준: 30
- 과매수 기준: 70


In [4]:
class RSIStrategy(TradingStrategy):
    """RSI 전략 - 지표 사전 계산 버전"""
    
    def __init__(self, rsi_period: int = 14, oversold: float = 30, overbought: float = 70):
        super().__init__(
            name="RSIStrategy",
            config={
                "rsi_period": rsi_period,
                "oversold": oversold,
                "overbought": overbought
            },
            position_size_pct=0.15,
            max_positions=5
        )
        self.rsi_period = rsi_period
        self.oversold = oversold
        self.overbought = overbought
        self.indicator_columns = ["rsi"]
        print(f"📈 RSI 전략 초기화 (기간: {rsi_period}일, 과매도: {oversold}, 과매수: {overbought})")
        
    def _compute_indicators_for_symbol(self, symbol_data: pl.DataFrame) -> pl.DataFrame:
        """심볼별 RSI 지표 계산"""
        # 시간순 정렬 확인
        data = symbol_data.sort("timestamp")
        
        # RSI 계산
        rsi = self.calculate_rsi(data["close"], self.rsi_period)
        
        print(f"   📊 {data['symbol'][0]} - RSI({self.rsi_period}) 계산 완료")
        
        # RSI 컬럼 추가
        return data.with_columns([
            rsi.alias("rsi")
        ])
    
    def generate_signals(self, data: MarketDataBatch) -> List[Order]:
        """신호 생성 - RSI 기반 매수/매도"""
        orders = []
        
        if not self.context:
            return orders
        
        for symbol in data.symbols:
            current_price = self.get_current_price(symbol, data)
            if not current_price:
                continue
            
            # 현재 RSI 값 조회
            rsi = self.get_indicator_value(symbol, "rsi", data)
            if rsi is None:
                continue
            
            current_positions = self.get_current_positions()
            
            # 과매도 구간 - 매수
            if rsi < self.oversold and symbol not in current_positions:
                portfolio_value = self.get_portfolio_value()
                quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
                
                if quantity > 0:
                    order = Order(
                        symbol=symbol,
                        side=OrderSide.BUY,
                        quantity=quantity,
                        order_type=OrderType.MARKET
                    )
                    orders.append(order)
                    print(f"🔵 과매도 - {symbol} 매수 신호! RSI: {rsi:.1f} < {self.oversold}")
            
            # 과매수 구간 - 매도
            elif rsi > self.overbought and symbol in current_positions and current_positions[symbol] > 0:
                order = Order(
                    symbol=symbol,
                    side=OrderSide.SELL,
                    quantity=current_positions[symbol],
                    order_type=OrderType.MARKET
                )
                orders.append(order)
                print(f"🔴 과매수 - {symbol} 매도 신호! RSI: {rsi:.1f} > {self.overbought}")
        
        return orders

# 전략 인스턴스 생성 및 테스트
rsi_strategy = RSIStrategy(rsi_period=14, oversold=25, overbought=75)
print(f"📋 전략명: {rsi_strategy.name}")
print(f"⚙️ 설정: {rsi_strategy.config}")
print(f"💰 포지션 크기: {rsi_strategy.position_size_pct * 100}%")
print(f"📊 필요 지표: {rsi_strategy.indicator_columns}")


📈 RSI 전략 초기화 (기간: 14일, 과매도: 25, 과매수: 75)
📋 전략명: RSIStrategy
⚙️ 설정: {'rsi_period': 14, 'oversold': 25, 'overbought': 75}
💰 포지션 크기: 15.0%
📊 필요 지표: ['rsi']


---

## 4️⃣ 랜덤 전략

**완전히 랜덤한 매매**를 수행하는 전략입니다. 다른 전략의 성과를 비교하기 위한 **벤치마크** 용도로 사용됩니다.

### 🎯 전략 로직
- 설정된 확률에 따라 거래 여부 결정
- 랜덤하게 심볼 선택
- 포지션 유무에 따라 매수/매도 랜덤 결정
- **"원숭이도 다트를 던져서 주식을 고를 수 있다"** 이론 검증


In [5]:
class RandomStrategy(TradingStrategy):
    """랜덤 전략 - 테스트 목적"""
    
    def __init__(self, trade_probability: float = 0.1):
        super().__init__(
            name="RandomStrategy",
            config={"trade_probability": trade_probability},
            position_size_pct=0.1,
            max_positions=3
        )
        self.trade_probability = trade_probability
        self.trade_count = 0
        print(f"🎲 랜덤 전략 초기화 (거래 확률: {trade_probability * 100}%)")
        
    def _compute_indicators_for_symbol(self, symbol_data: pl.DataFrame) -> pl.DataFrame:
        """랜덤 전략은 지표 불필요 - 원본 데이터 그대로 반환"""
        print(f"   🎲 {symbol_data['symbol'][0]} 심볼 데이터 준비 완료 (지표 계산 없음)")
        return symbol_data
    
    def generate_signals(self, data: MarketDataBatch) -> List[Order]:
        """신호 생성 - 랜덤"""
        import random
        
        orders = []
        
        if not self.context or random.random() > self.trade_probability:
            return orders
        
        # 랜덤하게 심볼 선택
        if data.symbols:
            symbol = random.choice(data.symbols)
            current_price = self.get_current_price(symbol, data)
            
            if current_price:
                # 랜덤하게 매수/매도 결정
                current_positions = self.get_current_positions()
                
                if symbol in current_positions and current_positions[symbol] > 0:
                    # 포지션이 있으면 50% 확률로 매도
                    if random.random() > 0.5:
                        order = Order(
                            symbol=symbol,
                            side=OrderSide.SELL,
                            quantity=current_positions[symbol],
                            order_type=OrderType.MARKET
                        )
                        orders.append(order)
                        self.trade_count += 1
                        print(f"🎯 랜덤 매도 - {symbol} (거래 #{self.trade_count})")
                else:
                    # 포지션이 없으면 매수
                    portfolio_value = self.get_portfolio_value()
                    quantity = self.calculate_position_size(symbol, current_price, portfolio_value)
                    
                    if quantity > 0:
                        order = Order(
                            symbol=symbol,
                            side=OrderSide.BUY,
                            quantity=quantity,
                            order_type=OrderType.MARKET
                        )
                        orders.append(order)
                        self.trade_count += 1
                        print(f"🎯 랜덤 매수 - {symbol} (거래 #{self.trade_count})")
        
        return orders

# 전략 인스턴스 생성 및 테스트
random_strategy = RandomStrategy(trade_probability=0.05)  # 5% 확률로 거래
print(f"📋 전략명: {random_strategy.name}")
print(f"⚙️ 설정: {random_strategy.config}")
print(f"💰 포지션 크기: {random_strategy.position_size_pct * 100}%")
print(f"🎲 거래 확률: {random_strategy.trade_probability * 100}%")


🎲 랜덤 전략 초기화 (거래 확률: 5.0%)
📋 전략명: RandomStrategy
⚙️ 설정: {'trade_probability': 0.05}
💰 포지션 크기: 10.0%
🎲 거래 확률: 5.0%


---

## 📊 전략 비교 및 실행

지금까지 살펴본 4가지 전략의 특징을 비교해보겠습니다.

### 전략별 특징 비교


In [6]:
import pandas as pd
from IPython.display import display, HTML

# 전략 비교표 생성
strategies_comparison = {
    '전략명': ['Buy & Hold', 'SMA Cross', 'RSI', 'Random'],
    '타입': ['추세추종', '추세추종', '평균회귀', '랜덤'],
    '포지션크기': ['100%', '20%', '15%', '10%'],
    '최대포지션': [10, 5, 5, 3],
    '주요지표': ['없음', 'SMA(10,30)', 'RSI(14)', '없음'],
    '거래빈도': ['매우낮음', '낮음', '중간', '랜덤'],
    '장점': ['단순함, 낮은수수료', '트렌드 포착', '변동성 활용', '편향 없음'],
    '단점': ['하락장 취약', '횡보장 취약', '강한추세시 불리', '수익성 없음']
}

df_comparison = pd.DataFrame(strategies_comparison)

print("📊 전략 비교표")

# Jupyter Notebook에서 예쁘게 출력
display(df_comparison)


📊 전략 비교표


,전략명,타입,포지션크기,최대포지션,주요지표,거래빈도,장점,단점
0,Buy & Hold,추세추종,100%,10,없음,매우낮음,"단순함, 낮은수수료",하락장 취약
1,SMA Cross,추세추종,20%,5,"SMA(10,30)",낮음,트렌드 포착,횡보장 취약
2,RSI,평균회귀,15%,5,RSI(14),중간,변동성 활용,강한추세시 불리
3,Random,랜덤,10%,3,없음,랜덤,편향 없음,수익성 없음


### 💡 전략 선택 가이드

| 시장 상황 | 추천 전략 | 이유 |
|----------|-----------|------|
| **강한 상승 추세** | Buy & Hold | 지속적인 상승에서 최대 수익 |
| **변동성 높은 횡보** | RSI | 과매도/과매수 구간 활용 |
| **트렌드 전환 많음** | SMA Cross | 트렌드 변화 민감하게 포착 |
| **시장 방향 불명** | Random | 다른 전략 성과 비교 기준 |

### 🎯 실제 백테스팅 예제

아래는 이러한 전략들을 실제로 백테스팅 엔진에서 사용하는 예제입니다:


In [7]:
# 백테스팅 실행 예제 (실제 실행을 위해서는 데이터와 엔진 설정 필요)

def demo_strategy_usage():
    """전략 사용법 데모"""
    
    print("🚀 QuantBT 전략 사용 예제")
    print("=" * 50)
    
    # 1. 전략들 생성
    strategies = {
        'conservative': BuyAndHoldStrategy(),
        'trend_following': SimpleMovingAverageCrossStrategy(short_window=5, long_window=20),
        'mean_reversion': RSIStrategy(rsi_period=14, oversold=30, overbought=70),
        'benchmark': RandomStrategy(trade_probability=0.02)
    }
    
    # 2. 각 전략의 기본 정보 출력
    for strategy_type, strategy in strategies.items():
        print(f"\n📈 {strategy_type.upper()}:")
        print(f"   이름: {strategy.name}")
        print(f"   포지션 크기: {strategy.position_size_pct * 100}%")
        print(f"   최대 포지션: {strategy.max_positions}")
        if hasattr(strategy, 'indicator_columns') and strategy.indicator_columns:
            print(f"   필요 지표: {', '.join(strategy.indicator_columns)}")
    
    print("\n✅ 모든 전략이 성공적으로 초기화되었습니다!")
    print("\n💡 실제 백테스팅을 위해서는 다음이 필요합니다:")
    print("   - 데이터 프로바이더 (CSV, Upbit 등)")
    print("   - 백테스트 엔진 설정")
    print("   - 백테스트 설정 (기간, 초기자본 등)")
    
    return strategies

# 데모 실행
demo_strategies = demo_strategy_usage()


🚀 QuantBT 전략 사용 예제
🏠 바이 앤 홀드 전략이 초기화되었습니다.
📈 이동평균 교차 전략 초기화 (단기: 5일, 장기: 20일)
📈 RSI 전략 초기화 (기간: 14일, 과매도: 30, 과매수: 70)
🎲 랜덤 전략 초기화 (거래 확률: 2.0%)

📈 CONSERVATIVE:
   이름: BuyAndHoldStrategy
   포지션 크기: 100.0%
   최대 포지션: 10

📈 TREND_FOLLOWING:
   이름: SimpleMovingAverageCrossStrategy
   포지션 크기: 20.0%
   최대 포지션: 5
   필요 지표: sma_5, sma_20

📈 MEAN_REVERSION:
   이름: RSIStrategy
   포지션 크기: 15.0%
   최대 포지션: 5
   필요 지표: rsi

📈 BENCHMARK:
   이름: RandomStrategy
   포지션 크기: 10.0%
   최대 포지션: 3

✅ 모든 전략이 성공적으로 초기화되었습니다!

💡 실제 백테스팅을 위해서는 다음이 필요합니다:
   - 데이터 프로바이더 (CSV, Upbit 등)
   - 백테스트 엔진 설정
   - 백테스트 설정 (기간, 초기자본 등)


---

## 🚀 실제 데이터로 전략 실행해보기

이제 실제 업비트 데이터를 사용하여 우리가 만든 전략들을 테스트해보겠습니다!

### 📊 테스트 설정
- **데이터 소스**: 업비트 API
- **심볼**: KRW-BTC (비트코인)
- **시간프레임**: 1분봉
- **기간**: 최근 1일 (1440분)
- **초기 자본**: 1,000,000원


In [8]:
# 업비트 데이터 프로바이더와 백테스팅 엔진 임포트
import asyncio
from datetime import datetime, timedelta
import pandas as pd

# 올바른 모듈 경로로 임포트
from quantbt.infrastructure.data.upbit_provider import UpbitDataProvider
from quantbt.infrastructure.engine.simple_engine import SimpleBacktestEngine
from quantbt.infrastructure.brokers.simple_broker import SimpleBroker
from quantbt.core.value_objects.backtest_config import BacktestConfig  # 수정: value_objects에 위치
print("✅ 백테스팅 관련 모듈 임포트 성공!")


# 업비트 데이터 프로바이더 생성
upbit_provider = UpbitDataProvider(
    cache_dir="./data/upbit_cache",  # 캐시 디렉토리
    rate_limit_delay=0.1  # API 호출 간격 (초)
)

print("🏦 업비트 데이터 프로바이더 초기화 완료!")
print(f"📁 캐시 디렉토리: ./data/upbit_cache")
print(f"⏱️ API 호출 간격: 0.1초")


✅ 백테스팅 관련 모듈 임포트 성공!
🏦 업비트 데이터 프로바이더 초기화 완료!
📁 캐시 디렉토리: ./data/upbit_cache
⏱️ API 호출 간격: 0.1초


In [21]:
# 백테스팅 설정
end_date = datetime.now()
start_date = end_date - timedelta(days=1)  # 1일 전

config = BacktestConfig(
    symbols=["KRW-BTC"],
    start_date=start_date,
    end_date=end_date,
    timeframe="1m",  # 1분봉
    initial_cash=1000000.0,  # 100만원
    commission_rate=0.,  # 수수료 - 실제 전략에선 적절한 값을 넣어주세요.
    slippage_rate=0.     # 슬리피지 - 실제 전략에선 적절한 값을 넣어주세요.
)

print("⚙️ 백테스팅 설정 완료!")
print(f"📅 기간: {start_date.strftime('%Y-%m-%d %H:%M')} ~ {end_date.strftime('%Y-%m-%d %H:%M')}")
print(f"💰 초기 자본: {config.initial_cash:,}원")
print(f"📊 심볼: {', '.join(config.symbols)}")
print(f"⏰ 시간프레임: {config.timeframe}")
print(f"💸 수수료: {config.commission_rate * 100}% - 실제 전략에선 적절한 값을 넣어주세요.")
print(f"🎯 슬리피지: {config.slippage_rate * 100}% - 실제 전략에선 적절한 값을 넣어주세요.")


⚙️ 백테스팅 설정 완료!
📅 기간: 2025-06-04 14:28 ~ 2025-06-05 14:28
💰 초기 자본: 1,000,000.0원
📊 심볼: KRW-BTC
⏰ 시간프레임: 1m
💸 수수료: 0.0%
🎯 슬리피지: 0.0%


In [22]:
# 업비트에서 데이터 가져오기
async def fetch_upbit_data():
    """업비트에서 BTC 1분봉 데이터 가져오기"""
    print("📡 업비트에서 데이터 가져오는 중...")
    
    try:
        # 데이터 로드
        data = await upbit_provider.load_data(
            symbols=config.symbols,
            start_date=config.start_date,
            end_date=config.end_date,
            timeframe=config.timeframe
        )
        
        print(f"✅ 데이터 로드 완료!")
        print(f"📊 데이터 크기: {len(data)} 행")
        print(f"📅 데이터 기간: {data['timestamp'].min()} ~ {data['timestamp'].max()}")
        
        # 데이터 미리보기
        print("\n📋 데이터 미리보기 (최근 5개):")
        display(data.tail().to_pandas())
        
        return data
        
    except Exception as e:
        print(f"❌ 데이터 로드 실패: {e}")
        raise

# 비동기 함수 실행
btc_data = await fetch_upbit_data()


📡 업비트에서 데이터 가져오는 중...
❌ 데이터 로드 실패: 'UpbitDataProvider' object has no attribute 'load_data'


AttributeError: 'UpbitDataProvider' object has no attribute 'load_data'

In [ ]:
# 백테스팅 실행 함수
async def run_strategy_backtest(strategy, strategy_name):
    """전략 백테스팅 실행"""
    print(f"\n🚀 {strategy_name} 백테스팅 시작...")
    
    try:
        # 브로커 생성
        broker = SimpleBroker(
            initial_cash=config.initial_cash,
            commission_rate=config.commission_rate,
            slippage_rate=config.slippage_rate
        )
        
        # 백테스트 엔진 생성 및 설정
        engine = SimpleBacktestEngine()
        engine.set_strategy(strategy)
        engine.set_data_provider(upbit_provider)
        engine.set_broker(broker)
        
        # 백테스팅 실행
        result = await engine.run(config)
        
        print(f"✅ {strategy_name} 백테스팅 완료!")
        
        # 결과 요약 출력
        print(f"📊 {strategy_name} 결과 요약:")
        print(f"   💰 최종 포트폴리오 가치: {result.final_portfolio_value:,.0f}원")
        print(f"   📈 총 수익률: {result.total_return_pct:.2f}%")
        print(f"   📉 최대 낙폭: {result.max_drawdown_pct:.2f}%")
        print(f"   🔄 총 거래 수: {result.total_trades}")
        if result.total_trades > 0:
            print(f"   🎯 승률: {result.win_rate_pct:.1f}%")
        
        return result
        
    except Exception as e:
        print(f"❌ {strategy_name} 백테스팅 실패: {e}")
        return None

# 테스트할 전략들 준비 (1분봉에 맞게 파라미터 조정)
test_strategies = {
    "Buy & Hold": BuyAndHoldStrategy(),
    "SMA Cross (빠른)": SimpleMovingAverageCrossStrategy(short_window=5, long_window=15),  # 1분봉용 짧은 기간
    "RSI (민감)": RSIStrategy(rsi_period=14, oversold=35, overbought=65),  # 1분봉용 덜 극단적 기준
}

print("🎯 테스트 전략 목록:")
for name, strategy in test_strategies.items():
    print(f"   - {name}: {strategy.__class__.__name__}")
    if hasattr(strategy, 'config'):
        print(f"     설정: {strategy.config}")


In [ ]:
# 모든 전략 백테스팅 실행
results = {}

for strategy_name, strategy in test_strategies.items():
    result = await run_strategy_backtest(strategy, strategy_name)
    if result:
        results[strategy_name] = result

print(f"\n🎉 모든 백테스팅 완료! 총 {len(results)}개 전략 테스트됨")


In [ ]:
# 결과 비교표 생성
if results:
    comparison_data = []
    
    for strategy_name, result in results.items():
        comparison_data.append({
            '전략명': strategy_name,
            '최종가치(원)': f"{result.final_portfolio_value:,.0f}",
            '수익률(%)': f"{result.total_return_pct:.2f}",
            '최대낙폭(%)': f"{result.max_drawdown_pct:.2f}",
            '총거래수': result.total_trades,
            '승률(%)': f"{result.win_rate_pct:.1f}" if result.total_trades > 0 else "N/A"
        })
    
    results_df = pd.DataFrame(comparison_data)
    
    print("🏆 백테스팅 결과 비교")
    print("=" * 80)
    display(results_df)
    print("=" * 80)
    
    # 최고 성과 전략 찾기
    best_strategy = max(results.items(), key=lambda x: x[1].total_return_pct)
    worst_strategy = min(results.items(), key=lambda x: x[1].total_return_pct)
    
    print(f"\n🥇 최고 성과: {best_strategy[0]} ({best_strategy[1].total_return_pct:.2f}%)")
    print(f"🥉 최저 성과: {worst_strategy[0]} ({worst_strategy[1].total_return_pct:.2f}%)")
    
    # 성과 차이 계산
    performance_gap = best_strategy[1].total_return_pct - worst_strategy[1].total_return_pct
    print(f"📊 성과 차이: {performance_gap:.2f}%포인트")
    
else:
    print("❌ 백테스팅 결과가 없습니다.")
